# CKO JAR Revision

In [1]:
import wrds, pandas as pd, numpy as np

In [2]:
import rpy2.rinterface

## Review TNIC-3 data

In [3]:
db = wrds.Connection(wrds_username="yaera")

Loading library list...
Done


In [4]:
tnic = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/tnic3_data.txt', delimiter='\t', header=0)

In [5]:
tnic.dropna(inplace=True)

Convert _GVKEY_ into strings

In [6]:
tnic['gvkey1'] = tnic['gvkey1'].apply(lambda x: str(x).zfill(6))
tnic['gvkey2'] = tnic['gvkey2'].apply(lambda x: str(x).zfill(6))

In [7]:
tnic.set_index(["year", "gvkey1", "gvkey2"], inplace=True, verify_integrity=True)

Calculate number of competitors by year and firm

In [8]:
industry = pd.DataFrame(tnic.groupby(level=['year','gvkey1']).size())
industry.rename(columns={0:"comp"}, inplace=True)

Check firms with less than 10 competitors

In [9]:
industry[industry["comp"] < 10] # 24997 firm-year observations

comp
year gvkey1      
1995 001004     1
1996 001038     9
     001043     2
     001050     4
     001072     9
...           ...
2017 264416     9
     272699     8
     282553     3
     315318     2
     316056     1

[24997 rows x 1 columns]

## Import previously constructed datasets

### Materiality of M&A

In [10]:
material = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/materiality.csv')

In [11]:
material['gvkey1'] = material['gvkey1'].apply(lambda x: str(x).zfill(6))
material.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

### M&A Disclosure

In [12]:
disc = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/disc.csv')

In [13]:
disc.rename(columns={"GVKEY":"gvkey1", "FYEAR":"year"}, inplace=True)
disc['gvkey1'] = disc['gvkey1'].apply(lambda x: str(x).zfill(6))
disc.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

In [14]:
merge = disc.join(material)[['DATADATE', 'CIK', 'TGTAT_ACQAT', 'TGTDVAL_ACQAT', 'MD_A', 'PROFORMA']].sort_index()

### SDC and Compustat Link File

The link file is from [Michael Ewens](https://github.com/michaelewens/SDC-to-Compustat-Mapping.git). Cite papers below.

```
@article{phillips2013r,
  title={R\&D and the Incentives from Merger and Acquisition Activity},
  author={Phillips, Gordon M and Zhdanov, Alexei},
  journal={The Review of Financial Studies},
  volume={26},
  number={1},
  pages={34--78},
  year={2013},
  publisher={Society for Financial Studies}
  }
 ```

```
@article{ewensPetersWang2018,
 title={Acquisition prices and the measurement of intangible capital},
 author={Ewens, Michael and Peters, Ryan and Wang, Sean},
 journal={Working Paper}
 year={2018}
 }
```

In [52]:
sdc_link = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/dealnum_to_gvkey.csv')
sdc_link['agvkey'] = sdc_link['agvkey'].apply(lambda x: str(int(x)).zfill(6) if pd.notnull(x) else x)
sdc_link['tgvkey'] = sdc_link['tgvkey'].apply(lambda x: str(int(x)).zfill(6) if pd.notnull(x) else x)
sdc_link['DealNumber'] = sdc_link['DealNumber'].apply(int)
sdc_link.set_index('DealNumber', inplace=True, verify_integrity=True)
sdc_link.sort_index(inplace=True)

In [ ]:
# ma_details_desc = db.describe_table('sdc', 'ma_details').sort_values('name')
# with pd.option_context('display.max_rows', None):
#     print(ma_details_desc)

|     Variable | Description                    |
|:------------:|:-------------------------------|
|bookvalue     |Target Book Value (\$mil)       |
|compete       |Competing Bidder (Y/N)          |
|competecode   |Competing Bid Deal Code         |  
|dateann       |Date Announced                  |
|dateannest    |_dateann_ is estimated (Y/N)    | 
|dateeff       |Date Effective                  | 
|ebitltm       |Target EBIT LTM (\$mil)         |
|pct_cash      |Percentage of consideration paid in cash|
|pct_other|Percentage of consideration paid in other then cash or stock|
|pct_stk|Percentage of consideration paid in stock|
|pct_unknown|Percentage of consideration which is unknown|
|ptincltm|Target Pre-Tax Income LTM (\$mil)|
|salesltm|Target Sales LTM (\$mil)|
|rankval|Ranking Value incl Net Debt of Target (\$mil)|

Run sql query below on _WRDS_

In [18]:
# import wrds
# sdc_query = """
# select master_deal_no as DealNumber, 
#         bookvalue, 
#         compete, 
#         competecode, 
#         dateann, 
#         dateannest, 
#         dateeff, 
#         ebitltm, 
#         pct_cash,
#         pct_other,
#         pct_stk,
#         pct_unknown,
#         ptincltm,
#         salesltm,
#         rankval
# from sdc.ma_details
# where dateeff is not null 
# """
# # and master_deal_no in %(deal_no)s
# sdc = db.raw_sql(sdc_query, date_cols=['dateann', 'dateeff'])
# sdc.to_pickle('/home/upenn/yaera/sdc.pkl')

In [57]:
sdc = pd.read_pickle('/Users/ohn0000/Project/cko/0_data/external/sdc.pkl')
sdc['dealnumber'] = sdc['dealnumber'].apply(int)

In [58]:
sdc_sub = pd.merge(sdc_link, sdc, left_index=True, right_on='dealnumber').drop('dealnumber', axis='columns')
sdc_sub.index.name = "DealNumber"